In [1]:
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))
c.head(3)

,date,location,new_cases,new_deaths,total_cases,total_deaths
0,2019-12-31,Afghanistan,0,0,0,0
1,2020-01-01,Afghanistan,0,0,0,0
2,2020-01-02,Afghanistan,0,0,0,0


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv")

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-26,France,125866.0,0.0,23080.0,0.0,0.0,0.0
0,2020-04-26,China,84014.0,0.0,4671.0,0.0,0.0,0.0
0,2020-04-26,Italy,198300.0,0.0,26788.0,0.0,0.0,0.0
0,2020-04-26,Spain,224483.0,0.0,22914.0,0.0,0.0,0.0
0,2020-04-26,United States,962705.0,0.0,54700.0,0.0,0.0,0.0
0,2020-04-26,World,2926424.0,0.0,206213.0,0.0,0.0,0.0
0,2020-04-26,United Kingdom,153730.0,0.0,21110.0,0.0,0.0,0.0
0,2020-04-26,Germany,155883.0,0.0,5764.0,0.0,0.0,0.0
0,2020-04-26,Iran,90479.0,0.0,5738.0,0.0,0.0,0.0
0,2020-04-26,Turkey,110305.0,0.0,2807.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-04-27,France,125866.0,124575.0,23080.0,22856.0,1291.0,224.0
0,2020-04-27,China,84014.0,84199.0,4671.0,4639.0,-185.0,32.0
0,2020-04-27,Italy,198300.0,197675.0,26788.0,26644.0,625.0,144.0
0,2020-04-27,Spain,224483.0,207634.0,22914.0,23190.0,16849.0,-276.0
0,2020-04-27,United States,962705.0,965910.0,54700.0,54876.0,-3205.0,-176.0
0,2020-04-27,World,2926424.0,2914507.0,206213.0,205923.0,11917.0,290.0
0,2020-04-27,United Kingdom,153730.0,152840.0,21110.0,20732.0,890.0,378.0
0,2020-04-27,Germany,155883.0,155193.0,5764.0,5750.0,690.0,14.0
0,2020-04-27,Iran,90479.0,90481.0,5738.0,5710.0,-2.0,28.0
0,2020-04-27,Turkey,110305.0,110130.0,2807.0,2805.0,175.0,2.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")